In [4]:
from utils import (
    download_file,
    parsear_archivo,
    random_path,
    calcular_distancias,
    cumple_capacidad,
    dist
)
from tqdm.notebook import tqdm as tqdm

In [5]:
download_file("https://modelosuno.okapii.com/Problemas/problema2022/problema_uno.txt", "problema_uno.txt", override=True)

  0%|          | 0.00/4.27k [00:00<?, ?iB/s]

In [6]:
CAPACIDAD, DIMENSION, DEMANDAS, TIPO_ARISTA, COORDS = parsear_archivo("problema_uno.txt")

In [7]:
%%time
costos = []
caminos = []

for initial in tqdm(range(1, DIMENSION+1)):
    if DEMANDAS[initial] < 0:
        continue
    seq = [initial]
    capacities = [DEMANDAS[initial]]

    while len(seq) < DIMENSION:
        last = seq[-1]
        last_capacity = capacities[-1]

        candidates = [k for k, v in DEMANDAS.items() if 0 <= last_capacity + v <= CAPACIDAD and k not in seq]

        best_candidate = None
        best_distance = None
        for candidate in candidates:
            d = dist(candidate, last, COORDS)
            if best_distance is None or d < best_distance:
                best_candidate = candidate
                best_distance = d

        seq.append(best_candidate)
        capacities.append(last_capacity + DEMANDAS[best_candidate])
    
    costos.append(calcular_distancias(seq, COORDS))
    caminos.append(seq)

  0%|          | 0/150 [00:00<?, ?it/s]

CPU times: user 1.63 s, sys: 8.16 ms, total: 1.63 s
Wall time: 1.63 s


In [8]:
idx_minimo = costos.index(min(costos))
camino_minimo = caminos[idx_minimo]

print(idx_minimo)
print(cumple_capacidad(camino_minimo, DEMANDAS, CAPACIDAD))
print(calcular_distancias(camino_minimo, COORDS))

31
True
8428.020072199306


In [16]:
%%time
path = camino_minimo

improvement = True
best_path = path
best_cost = calcular_distancias(best_path, COORDS)
capacity_ok = cumple_capacidad(best_path, DEMANDAS, CAPACIDAD)

while improvement:
    improvement = False
    for i in range(len(best_path) - 1):
        for k in range(i + 1, len(best_path)):
            new_path = best_path[0:i]
            new_path.extend(reversed(best_path[i:k + 1]))
            new_path.extend(best_path[k + 1:])
            new_cost = calcular_distancias(new_path, COORDS)
            
            if capacity_ok:
                if not cumple_capacidad(new_path, DEMANDAS, CAPACIDAD):
                    continue
            elif cumple_capacidad(new_path, DEMANDAS, CAPACIDAD):
                capacity_ok = True

            if new_cost < best_cost:
                print(f"Improved from {best_cost} for {new_cost}")
                best_cost = new_cost
                best_path = new_path
                improvement = True
                break  # improvement found, return to the top of the while loop
        if improvement:
            break

Improved from 8428.020072199306 for 8414.931779851968
Improved from 8414.931779851968 for 8398.048417209699
Improved from 8398.048417209699 for 8385.504330749312
Improved from 8385.504330749312 for 8361.648400310136
Improved from 8361.648400310136 for 8345.858858849702
Improved from 8345.858858849702 for 8330.209236513441
Improved from 8330.209236513441 for 8261.132283239298
Improved from 8261.132283239298 for 8208.23080589449
Improved from 8208.23080589449 for 8183.621946889229
Improved from 8183.621946889229 for 8106.021273276528
Improved from 8106.021273276528 for 8060.453465847144
Improved from 8060.453465847144 for 8004.496975088792
Improved from 8004.496975088792 for 7992.057706091682
Improved from 7992.057706091682 for 7985.646752289846
Improved from 7985.646752289846 for 7979.427534222015
Improved from 7979.427534222015 for 7979.419172637662
Improved from 7979.419172637662 for 7948.827987532666
Improved from 7948.827987532666 for 7927.486321845482
Improved from 7927.48632184548

In [22]:
cumple_capacidad(best_path, DEMANDAS, CAPACIDAD)

True

In [23]:
calcular_distancias(best_path, COORDS)

7815.878420373433

In [24]:
seq_str = " ".join(map(str, best_path))
print(seq_str)

80 14 122 77 23 38 32 131 67 43 109 134 54 92 90 46 138 51 20 25 141 58 55 65 85 132 50 137 102 114 99 19 108 70 135 86 29 81 110 47 139 40 53 12 24 118 116 101 41 57 117 129 27 31 123 74 13 106 91 119 68 128 45 71 64 44 150 115 4 104 22 62 149 125 21 78 15 133 111 105 33 126 52 16 59 79 121 88 94 10 113 3 48 63 73 76 87 1 98 103 34 30 84 7 8 89 96 35 124 93 97 100 5 107 95 82 143 142 18 75 26 146 56 83 2 37 9 6 28 42 120 127 69 36 61 11 148 130 39 60 66 17 140 136 112 145 144 147 49 72


In [25]:
with open("entrega_greedy_plus_2opt.txt", "w") as f:
    f.write(seq_str)